In [ ]:
!pip install --upgrade google-cloud-bigquery[bqstorage,pandas]

In [1]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


### Let's look at the training data

In [3]:
%%bigquery retail_dataset

SELECT
  EXTRACT (dayofweek
  FROM
    date) AS dayofweek,
  EXTRACT (hour
  FROM
    date) AS hour,
  EXTRACT (day
  FROM
    date) AS day,
  EXTRACT (month
  FROM
    date) AS month,
  deviceCategory,
  isMobile,
  operatingSystem,
  browser,
  country,
  city,
  trafficSource,
  trafficMedium,
  trafficCampaign,
  isFirstVisit,
  totalPageviews,
  totalTimeOnSite,
  productPagesViewed,
  addedToCart
FROM
  `kaggleworkshops.google_analytics_sample.train`

In [4]:
retail_dataset

,dayofweek,hour,day,month,deviceCategory,isMobile,operatingSystem,browser,country,city,trafficSource,trafficMedium,trafficCampaign,isFirstVisit,totalPageviews,totalTimeOnSite,productPagesViewed,addedToCart
0,1,0,2,4,tablet,True,iOS,Safari,United States,not available in demo dataset,(direct),(none),(not set),0,35,250,12,1
1,1,0,2,4,mobile,True,iOS,Safari,United States,San Francisco,(direct),(none),(not set),1,52,3171,6,1
2,1,0,2,4,tablet,True,iOS,Safari,United States,not available in demo dataset,(direct),(none),(not set),1,110,2443,20,1
3,1,0,2,4,tablet,True,iOS,Safari,Australia,not available in demo dataset,(direct),(none),(not set),1,9,198,4,1
4,1,0,2,4,tablet,True,iOS,Safari,United States,not available in demo dataset,(direct),(none),(not set),0,25,1024,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765702,3,0,16,5,desktop,False,Macintosh,Chrome,United States,San Francisco,google,organic,(not set),1,15,472,1,0
765703,3,0,16,5,desktop,False,Macintosh,Chrome,United States,not available in demo dataset,google,organic,(not set),1,16,379,0,1
765704,3,0,16,5,desktop,False,Macintosh,Chrome,United States,not available in demo dataset,(direct),(none),(not set),1,16,353,0,1
765705,3,0,16,5,desktop,False,Macintosh,Chrome,United States,Mountain View,(direct),(none),(not set),0,16,377,1,0


### Train an xgboost model

In [1]:
%%bigquery

CREATE OR REPLACE MODEL
  bqmlretail.boosted_tree_model OPTIONS( model_type='boosted_tree_classifier',
    INPUT_LABEL_COLS=["addedToCart"]) AS
SELECT
  EXTRACT (dayofweek
  FROM
    date) AS dayofweek,
  EXTRACT (hour
  FROM
    date) AS hour,
  EXTRACT (day
  FROM
    date) AS day,
  EXTRACT (month
  FROM
    date) AS month,
  deviceCategory,
  isMobile,
  operatingSystem,
  browser,
  country,
  city,
  trafficSource,
  trafficMedium,
  trafficCampaign,
  isFirstVisit,
  totalPageviews,
  totalTimeOnSite,
  productPagesViewed,
  addedToCart
FROM
  `kaggleworkshops.google_analytics_sample.train`;

""


### Get training info

In [37]:
%%bigquery
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL bqmlretail.boosted_tree_model)

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,20,0.092215,0.091216,0.3,81
1,0,19,0.092580,0.091480,0.3,41
2,0,18,0.093071,0.091898,0.3,57
3,0,17,0.093693,0.092412,0.3,82
4,0,16,0.094472,0.093189,0.3,41
5,0,15,0.095502,0.094243,0.3,62
6,0,14,0.096872,0.095483,0.3,63
7,0,13,0.098773,0.097251,0.3,14054
8,0,12,0.101249,0.099675,0.3,59
9,0,11,0.104630,0.103096,0.3,73


### Evaluate Model

In [36]:
BUCKET = 'gs://<bucket-name>/exported/xgboost'

In [31]:
%%bigquery 
SELECT
  *
FROM
  ML.EVALUATE (MODEL bqmlretail.boosted_tree_model)

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.690184,0.419776,0.958346,0.522042,0.091242,0.97138


### Export Model to GCS

In [41]:
!bq extract --destination_format ML_XGBOOST_BOOSTER -m bqmlretail.boosted_tree_model $BUCKET

FATAL Flags parsing error: flag --destination_format=XGBOOST_BOOSTER: value should be one of <CSV|NEWLINE_DELIMITED_JSON|AVRO|SAVED_MODEL>
Run 'bq.py help' to get help


In [39]:
!gsutil ls $BUCKET

gs://camus-bqml-retail/exported/xgboost/main.py
gs://camus-bqml-retail/exported/xgboost/model.bst
gs://camus-bqml-retail/exported/xgboost/xgboost_predictor-0.1.tar.gz
gs://camus-bqml-retail/exported/xgboost/assets/


### Copy model locally

In [42]:
!gsutil cp -r $BUCKET .

Copying gs://camus-bqml-retail/exported/xgboost/assets/10.txt...
Copying gs://camus-bqml-retail/exported/xgboost/assets/11.txt...                
Copying gs://camus-bqml-retail/exported/xgboost/assets/12.txt...                
Copying gs://camus-bqml-retail/exported/xgboost/assets/4.txt...                 
- [4 files][  4.2 KiB/  4.2 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://camus-bqml-retail/exported/xgboost/assets/5.txt...
Copying gs://camus-bqml-retail/exported/xgboost/assets/6.txt...                 
Copying gs://camus-bqml-retail/exported/xgboost/assets/7.txt...                 
Copying gs://camus-bqml-retail/exported/xgboost/assets/8.txt...                 
Copying gs://camus-bqml-retail/exported/xgboost/asse

### Unzip model

In [22]:
%pushd xgboost
!tar -xf xgboost_predictor-0.1.tar.gz

/home/jupyter/BQ demo/xgboost
/home/jupyter/BQ demo
popd -> ~/BQ demo


### Make a prediction

In [23]:
!python main.py '[{"dayofweek": 1,"hour": 0, "day": 28, "month": 8, "deviceCategory": "tablet", "isMobile": true, "operatingSystem": "iOS", "browser": "Safari", "country": "United States", "city": "San Jose", "trafficSource": "google", "trafficMedium": "organic", "trafficCampaign": "(not set)", "isFirstVisit": 1, "totalPageviews": 23, "totalTimeOnSite": 478, "productPagesViewed": 9 }]'

/home/jupyter/BQ demo/xgboost
[09:59:11] WARNING: /workspace/src/learner.cc:727: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility
[{'predicted_addedToCart': '0', 'addedToCart_values': ['1', '0'], 'addedToCart_probs': [0.39972978830337524, 0.6002702116966248]}]
